# 1. Pragmatic Ambiguous Sentences Dataset
- Extraction information of each Utterance

## 2) Extract emotion labeles from json

In [11]:
import os
import json
import pandas as pd

In [23]:
def parse_json_objects(unit_folder, json_path):
    """
    1) JSON 파일 로드
    2) data[frame_no][object_id] 순회
    3) object_id in [1, 2], label='person', text 필드 존재 시
       - frame, predicate, emotion, label, text 내 script, script_start, script_end, intent, strategy, morpheme
       - person_id
       - emotion 내 각 (valence, emotion, arousal) for image, sound, multimodal, text
    4) 결과를 (dict) 리스트 형태로 반환
    """

    records = []
    seen_text_triples = set()  # (script, script_start, script_end) 중복 체크용 집합

    if not os.path.isfile(json_path):
        print(f"[스킵] 파일 없음: {json_path}")
        return records

    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except UnicodeDecodeError as e:
        print(f"[에러] UTF-8로 디코딩할 수 없는 파일입니다: {json_path} - {str(e)}")
        return records

    # data 필드 (프레임별 객체)
    frame_dict = data.get("data", {})
    n_record = 0
    for frame_no, obj_dict in frame_dict.items():
        # obj_dict: 각 object_id -> object_info
        for object_id, info in obj_dict.items():
            # object_id가 '1' 또는 '2'만 추출 (문자열이므로 int 변환 후 비교 가능)
            try:
                obj_id_int = int(object_id)
            except:
                continue

            if obj_id_int not in [1, 2, 3,4, 5,6,7,8,9,10,11,203, 204, 259, 260,267,268,107,108,\
                                  21,22,97,98,255,256,38,37]: # 레이블링 에러 파일 때문에 person_id 203,204 추가 
                continue

            # label이 "person"인지 확인
            label = info.get("label", "")  # .lower()
            if label != "사람" and label != "person":
                continue

            # text 필드가 존재하는지 확인
            text_info = info.get("text", None)
            if not text_info:
                # text가 없으면 스킵
                continue
            
            # text 필드
            script = text_info.get("script", None)
            script_start = text_info.get("script_start", None) # 시작 프레임 
            script_end = text_info.get("script_end", None)  # 종료 프레임             
            
            # (script, script_start, script_end) 중복 확인
            text_key = (script, script_start, script_end)
            if text_key in seen_text_triples:
                # 이미 동일한 텍스트 구간이 기록되었다면 스킵
                continue
            seen_text_triples.add(text_key)
            n_record += 1

            intent = text_info.get("intent", None)
            strategy = text_info.get("strategy", None)
            morpheme = text_info.get("morpheme", None)  # 텍스트 인코더 입력 고려하여 남김 
            
            # data 객체 층위, 필요한 필드 추출
            clip_id = data.get("clip_id", None)
            actor = data.get("actor", None)
            situation = data.get("situation", None)
            category = data.get("category", None)
            nr_frame = data.get("nr_frame", None)  # 전체 프레임 수
            predicate = info.get("predicate", {})
            emotion = info.get("emotion", {})
            person_id = info.get("person_id", None)
            # emotion 
            # (image/sound/multimodal/text 중 image 제외한 각각의 valence, emotion, arousal)
            sound_emo = emotion.get("sound", {})
            multi_emo = emotion.get("multimodal", {})
            text_emo = emotion.get("text", {})
            # (emotion, valence, arousal) 추출
            sound_emotion = sound_emo.get("emotion", None)
            sound_valence = sound_emo.get("valence", None)
            sound_arousal = sound_emo.get("arousal", None)
            multimodal_emotion = multi_emo.get("emotion", None)
            multimodal_valence = multi_emo.get("valence", None)
            multimodal_arousal = multi_emo.get("arousal", None)
            text_emotion = text_emo.get("emotion", None)
            text_valence = text_emo.get("valence", None)
            text_arousal = text_emo.get("arousal", None)

            # record 생성
            file_index = f'{unit_folder}_{clip_id}_{n_record}'
            record = {
                "file_index": file_index,
                "clip_id": clip_id,
                "actor": actor, 
                "category": category,
                "nr_frame": nr_frame,                  
                "frame": frame_no,
                "object_id": object_id,
                "label": label,
                "predicate": predicate,   # dict로 저장
                "person_id": person_id,
                # text info
                "script": script,
                "script_start": script_start,
                "script_end": script_end,
                "intent": intent,
                "strategy": strategy,
                "morpheme": morpheme,
                # emotion - sound
                "sound_valence": sound_valence,
                "sound_emotion": sound_emotion,
                "sound_arousal": sound_arousal,
                # emotion - multimodal
                "multimodal_valence": multimodal_valence,
                "multimodal_emotion": multimodal_emotion,
                "multimodal_arousal": multimodal_arousal,
                # emotion - text
                "text_valence": text_valence,
                "text_emotion": text_emotion,
                "text_arousal": text_arousal
            }
            records.append(record)

    return records

In [7]:
def process_all_json(input_data_root_dir, output_path, list_unit_folder, clip_range):
    all_dfs = []
    for i, unit_name in enumerate(list_unit_folder):
        for clip_id in range(clip_range[i][0], clip_range[i][1]):  # for clip_id in range(1, 401):
            clip_folder = f"clip_{clip_id}"
            input_path = os.path.join(input_data_root_dir, unit_name, clip_folder)
            if not os.path.isdir(input_path):
                print(f"[스킵] 폴더 없음: {input_path}")
                continue
            json_path = os.path.join(input_path, f"clip_{clip_id}.json")   
            if not os.path.isfile(json_path):
                print(f"[스킵] JSON 파일 없음: {json_path}")
                continue
            data_records = parse_json_objects(unit_name, json_path)
            if not data_records:
                print(f"[완료] 발화 구간 없음: clip_{clip_id}")
                continue
            df = pd.DataFrame(data_records)
            df["unit_folder"] = unit_name
            print(f"[INFO] clip_{clip_id} -> 총 {len(df)}개 발화 구간 레코드 dataframe이 저장되었습니다.")
            all_dfs.append(df)  # 400개 clip의 df 저장  
        df_all_data = pd.concat(all_dfs, ignore_index=True)
        unit_dfs = df_all_data[df_all_data["unit_folder"] == unit_name].copy()
        save_path = os.path.join(output_path, f'{unit_name}.csv')
        unit_dfs.to_csv(save_path, index=False, encoding="utf-8-sig")
        print(f"[INFO] 400개 clip의 CSV 파일이 저장되었습니다: {save_path}")
    
    if not all_dfs:
        print("[INFO] 처리할 데이터가 없습니다.")
        return None
    df_all_data = pd.concat(all_dfs, ignore_index=True)
    merged_filename = "merged_data.csv"
    save_path = os.path.join(output_path, merged_filename)
    df_all_data.to_csv(save_path, index=False, encoding="utf-8-sig")
    print(f"[INFO] 모든 DataFrame이 합쳐진 CSV 파일이 저장되었습니다: {save_path}")
    
    return df_all_data

In [32]:
input_data_root_dir = 'G:/내 드라이브/aiffel/aiffelthon/multimodal/멀티모달 영상'
output_path = 'G:/내 드라이브/aiffel/aiffelthon/multimodal/labeled_data'
list_unit_folder = ['0001-0400', '0401-0800', '0801-1200', '1201-1600',\
                   '1601-2000', '2001-2400', '2401-2800', '2801-3200', '3201-3600', '3601-4000',\
                   '4001-4400', '4401-4800', '4801-5200', '5201-5600']

clip_range = [
    (1, 401),
    (401, 801),
    (801, 1201),
    (1201, 1601),
    (1601, 2001),
    (2001, 2401),
    (2401, 2801),
    (2801, 3201),
    (3201, 3601),
    (3601, 4001),
    (4001, 4401),
    (4401, 4801),
    (4801, 5201),
    (5201, 5601)
]


In [ ]:
# process_all_json(input_data_root_dir, output_path, list_unit_folder, clip_range)

# 누락 파일 추가 추출

In [34]:
# # 누락된 clip_id 목록
# missing_clip_ids = [
#     421, 743, 1052, 1444, 1445, 1462, 1463, 1925, 1926, 2050, 2124, 2281, 2282, 
#     2283, 2284, 3201, 3202, 3203, 3204, 3205, 3206, 3207, 3208, 3209, 3210, 
#     3211, 3212, 3213, 3214, 3215, 3216, 3217, 3218, 3219, 3220, 3221, 3222, 
#     3223, 3224, 3225, 3226, 3227, 3228, 3229, 3230, 3231, 3232, 3233, 3234, 
#     3235, 3236, 3237, 3238, 3239, 3240, 3241, 3242, 3243, 3244, 3245, 3246, 
#     3247, 3248, 3249, 3250, 3251, 3252, 3253, 3254, 3255, 3256, 3257, 3258, 
#     3259, 3260, 3261, 3262, 3263, 3264, 3265, 3266, 3267, 3268, 3269, 3270, 
#     3271, 3272, 3273, 3274, 3275, 3276, 3277, 3278, 3279, 3280, 3281, 3282, 
#     3283, 3284, 3285, 3286, 3287, 3288, 3289, 3290, 3291, 3292, 3293, 3294, 
#     3295, 3296, 3297, 3298, 3299, 3300, 3301, 3302, 3303, 3304, 3305, 3306, 
#     3307, 3308, 3309, 3310, 3311, 3312, 3313, 3314, 3315, 3316, 3317, 3318, 
#     3319, 3320, 3321, 3322, 3323, 3324, 3325, 3326, 3327, 3328, 3329, 3330, 
#     3331, 3332, 3333, 3334, 3335, 3336, 3337, 3338, 3339, 3340, 3341, 3342, 
#     3343, 3344, 3345, 3346, 3347, 3348, 3349, 3350, 3351, 3352, 3353, 3354, 
#     3355, 3356, 3357, 3358, 3359, 3360, 3361, 3362, 3363, 3364, 3365, 3366, 
#     3367, 3368, 3369, 3370, 3371, 3372, 3373, 3374, 3375, 3376, 3377, 3378, 
#     3379, 3380, 3381, 3382, 3383, 3384, 3385, 3386, 3387, 3388, 3389, 3390, 
#     3391, 3392, 3393, 3394, 3395, 3396, 3397, 3398, 3399, 3400, 3401, 3402, 
#     3403, 3404, 3405, 3406, 3407, 3408, 3409, 3410, 3411, 3412, 3413, 3414, 
#     3415, 3416, 3417, 3418, 3419, 3420, 3421, 3422, 3423, 3424, 3425, 3426, 
#     3427, 3428, 3429, 3430, 3431, 3432, 3433, 3434, 3435, 3436, 3437, 3438, 
#     3439, 3440, 3441, 3442, 3443, 3444, 3445, 3446, 3447, 3448, 3449, 3450, 
#     3451, 3452, 3453, 3454, 3455, 3456, 3457, 3458, 3459, 3460, 3461, 3462, 
#     3463, 3464, 3465, 3466, 3467, 3468, 3469, 3470, 3471, 3472, 3473, 3474, 
#     3475, 3476, 3477, 3478, 3479, 3480, 3481, 3482, 3483, 3484, 3485, 3486, 
#     3487, 3488, 3489, 3490, 3491, 3492, 3493, 3494, 3495, 3496, 3497, 3498, 
#     3499, 3500, 3501, 3502, 3503, 3504, 3505, 3506, 3507, 3508, 3509, 3510, 
#     3511, 3512, 3513, 3514, 3515, 3516, 3517, 3518, 3519, 3520, 3521, 3522, 
#     3523, 3524, 3525, 3526, 3527, 3528, 3529, 3530, 3531, 3532, 3533, 3534, 
#     3535, 3536, 3537, 3538, 3539, 3540, 3541, 3542, 3543, 3544, 3545, 3546, 
#     3547, 3548, 3549, 3550, 3551, 3552, 3553, 3554, 3555, 3556, 3557, 3558, 
#     3559, 3560, 3561, 3562, 3563, 3564, 3565, 3566, 3567, 3568, 3569, 3570, 
#     3571, 3572, 3573, 3574, 3575, 3576, 3577, 3578, 3579, 3580, 3581, 3582, 
#     3583, 3584, 3585, 3586, 3587, 3588, 3589, 3590, 3591, 3592, 3593, 3594, 
#     3595, 3596, 3597, 3598, 3599, 3600, 4052, 4053, 4054, 4055, 4056, 4296, 
#     4297, 4332, 4333, 4334, 4335, 4373, 4421, 4524, 4881, 5096, 5161, 5162, 
#     5163, 5164, 5165, 5166, 5167, 5168, 5169, 5170, 5321, 5322, 5323, 5324, 
#     5325, 5326, 5327
# ]

# print(len(missing_clip_ids))

448


In [36]:
# # unit_folder 목록
# list_unit_folder = [
#     '0001-0400', '0401-0800', '0801-1200', '1201-1600',
#     '1601-2000', '2001-2400', '2401-2800', '2801-3200', 
#     '3201-3600', '3601-4000', '4001-4400', '4401-4800', 
#     '4801-5200', '5201-5600'
# ]

# # clip_range 목록
# clip_range = [
#     (1, 401),
#     (401, 801),
#     (801, 1201),
#     (1201, 1601),
#     (1601, 2001),
#     (2001, 2401),
#     (2401, 2801),
#     (2801, 3201),
#     (3201, 3601),
#     (3601, 4001),
#     (4001, 4401),
#     (4401, 4801),
#     (4801, 5201),
#     (5201, 5601)
# ]

# # unit_folder과 clip_range를 매핑하기 위해 zip 사용
# unit_clip_mapping = list(zip(list_unit_folder, clip_range))

# # 누락된 clip_id가 속한 unit_folder와 clip_range를 저장할 집합(Set)
# missing_units = set()
# missing_ranges = set()

# # 각 누락된 clip_id에 대해 해당하는 unit_folder와 clip_range 찾기
# for clip_id in missing_clip_ids:
#     for unit, cr in unit_clip_mapping:
#         # clip_range는 시작값 이상, 끝값 미만으로 간주 (즉, [start, end))
#         if cr[0] <= clip_id < cr[1]:
#             missing_units.add(unit)
#             missing_ranges.add(cr)
#             break  # 한 clip_id는 하나의 unit_folder에만 속하므로 break

# # 필터링된 unit_folder와 clip_range를 리스트로 변환
# filtered_unit_folders = sorted(list(missing_units))
# filtered_clip_ranges = sorted(list(missing_ranges))

# # 결과 출력
# print("필터링된 unit_folder 목록:")
# for unit in filtered_unit_folders:
#     print(unit)

# print("\n필터링된 clip_range 목록:")
# for cr in filtered_clip_ranges:
#     print(cr)

필터링된 unit_folder 목록:
0401-0800
0801-1200
1201-1600
1601-2000
2001-2400
3201-3600
4001-4400
4401-4800
4801-5200
5201-5600

필터링된 clip_range 목록:
(401, 801)
(801, 1201)
(1201, 1601)
(1601, 2001)
(2001, 2401)
(3201, 3601)
(4001, 4401)
(4401, 4801)
(4801, 5201)
(5201, 5601)


In [9]:
# filtered_unit_folders = ['3201-3600']
# filtered_clip_ranges = [(3201,3601)]

In [17]:
# input_data_root_dir = 'G:/내 드라이브/aiffel/aiffelthon/multimodal/멀티모달 영상'
# output_path = 'G:/내 드라이브/aiffel/aiffelthon/multimodal/labeled_data/re-labeled_data'
# df = process_all_json(input_data_root_dir, output_path, filtered_unit_folders, filtered_clip_ranges)

[INFO] clip_3201 -> 총 15개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3202 -> 총 18개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3203 -> 총 11개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3204 -> 총 11개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3205 -> 총 15개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3206 -> 총 18개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3207 -> 총 14개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3208 -> 총 8개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3209 -> 총 19개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3210 -> 총 17개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3211 -> 총 20개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3212 -> 총 16개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3213 -> 총 17개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3214 -> 총 21개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3215 -> 총 15개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3216 -> 총 16개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3217 -> 총 17개 발화 구간 레코드 dataframe이 저장되었습니다.
[INFO] clip_3218 -> 총 16개 발화 구간 레코드 dataframe이 저장

In [ ]:
# filtered_unit_folders = ['0401-0800', \
#                         '0401-0800', \
#                         '0801-1200', \
#                         '1201-1600', \
#                         '1601-2000', \
#                         '2001-2400', \
#                         '2001-2400', \
#                         '2001-2400', \
#                         '4001-4400', \
#                         '4401-4800', \
#                         '4401-4800', \
#                         '4801-5200', \
#                         '4801-5200', \
#                         '4801-5200', \
#                         '5201-5600']

# filtered_clip_ranges = [(421,423), \
#                         (743,745), \
#                         (1052,1054), \
#                         (1444,1464), \
#                         (1925,1927), \
#                         (2050,2052), \
#                         (2124,2126), \
#                         (2281,2285), \
#                         (4052,4374), \
#                         (4421,4423), \
#                         (4524,4526), \
#                         (4881,4883), \
#                         (5096,5098), \
#                         (5161,5171), \
#                         (5321,5328)]

In [39]:
# filtered_unit_folders = ['2001-2400','2001-2400']
# filtered_clip_ranges = [(2124,2125),(2281,2285)]

In [29]:
# filtered_unit_folders = ['4001-4400', '4001-4400', '4001-4400', '4001-4400']
# filtered_clip_ranges = [(4052, 4057), (4296, 4298), (4332, 4336), (4373, 4374)]

In [41]:
# filtered_unit_folders = ['4801-5200', '4801-5200']
# filtered_clip_ranges = [(4881, 4882), (5161, 5171)]

In [43]:
# filtered_unit_folders = ['5201-5600']
# filtered_clip_ranges = [(5321,5328)]

In [29]:
# filtered_unit_folders = ['4801-5200']
# filtered_clip_ranges = [(5096,5097)]

In [31]:
# filtered_unit_folders = ['2001-2400']
# filtered_clip_ranges = [(2050, 2051)]

In [33]:
# filtered_unit_folders = ['3201-3600']
# filtered_clip_ranges = [(3251 , 3252)]

In [35]:
# filtered_unit_folders = ['0401-0800', '0401-0800', '0801-1200', '1201-1600', '1201-1600', '1601-2000',\
#                          '4401-4800', '4401-4800']
# filtered_clip_ranges = [(421,422), (743,744), (1052,1053), (1444, 1445), (1462, 1463), (1925, 1926),\
#                         (4421,4422), (4524,4525)]

In [37]:
# input_data_root_dir = 'G:/내 드라이브/aiffel/aiffelthon/multimodal/멀티모달 영상'
# output_path = 'G:/내 드라이브/aiffel/aiffelthon/multimodal/labeled_data/re-labeled_data'
# df = process_all_json(input_data_root_dir, output_path, filtered_unit_folders, filtered_clip_ranges)